In [1]:
import grpc
from apis import PredictRequest, PreditionService, ModelSpec

/home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jagan/.virtualenvs/tfs-cl

In [2]:
model_spec = ModelSpec(name='mnist', version_label='canary')

In [3]:
model_spec

name: "mnist"
version_label: "canary"

In [4]:
model_spec.version = 575

In [5]:
model_spec

name: "mnist"
version {
  value: 575
}

In [6]:
from tensorflow.core.framework import tensor_pb2

In [7]:
empty_proto = tensor_pb2.TensorProto()

In [8]:
predict_request = PredictRequest(model_spec=model_spec, inputs={'k':empty_proto})

In [9]:
predict_request

model_spec {
  name: "mnist"
  version {
    value: 575
  }
}
inputs {
  key: "k"
  value {
  }
}

In [10]:
predict_request.inputs = {'j':[1,2,3]}

In [15]:
predict_request.output_filter.extend(['num', 'num_classes'])

In [16]:
predict_request

model_spec {
  name: "mnist"
  version {
    value: 575
  }
}
inputs {
  key: "j"
  value {
    dtype: DT_INT32
    tensor_shape {
      dim {
        size: 3
      }
    }
    tensor_content: "\001\000\000\000\002\000\000\000\003\000\000\000"
  }
}
inputs {
  key: "k"
  value {
  }
}
output_filter: "num"
output_filter: "num_classes"
output_filter: "num"
output_filter: "num_classes"

In [54]:
isinstance(model_spec, predict_request.__class__.__bases__)

True

In [55]:
predict_request.__class__.__bases__

(base._Message,)

In [56]:
import numpy as np

In [57]:
predict_request

model_spec {
  name: "mnist"
  version {
    value: 575
  }
}
inputs {
  key: "j"
  value {
    dtype: DT_INT32
    tensor_shape {
      dim {
        size: 3
      }
    }
    tensor_content: "\001\000\000\000\002\000\000\000\003\000\000\000"
  }
}
inputs {
  key: "k"
  value {
  }
}
output_filter: "num_detections"
output_filter: "num_classes"

In [58]:
import os

In [59]:
model_spec.to_pb('./save/model_spec.pb')

In [60]:
model_spec.version_label = 'sdt'

In [61]:
model_spec

name: "mnist"
version_label: "sdt"

In [62]:
model_spec.from_pb('./save/model_spec.pb')

In [63]:
model_spec

name: "mnist"
version {
  value: 575
}

In [64]:
model_spec.to_text('./save/model_spec.pbtxt')

In [65]:
! pwd

/home/jagan/Jagan/Projects/Serving/simple-tfs-client


In [75]:
model_spec.version_label = 'new'

In [79]:
model_spec

name: "mnist"
version {
  value: 575
}

In [84]:
new_model_spec = ModelSpec()

In [85]:
new_model_spec

In [86]:
new_model_spec.from_text('./save/model_spec.pbtxt')

In [87]:
new_model_spec

name: "mnist"
version {
  value: 575
}

In [88]:
model_spec.version_label = 's'

In [89]:
model_spec

name: "mnist"
version_label: "s"

In [1]:
from tensorflow_serving.sources.storage_path import file_system_storage_path_source_pb2

In [2]:
model_version_policy = file_system_storage_path_source_pb2.FileSystemStoragePathSourceConfig.ServableVersionPolicy()

In [5]:
model_version_policy.WhichOneof('policy_choice')

In [8]:
model_version_policy.latest.num_versions = 1

In [9]:
model_version_policy.WhichOneof('policy_choice')

'latest'

In [21]:
model_version_policy.specific.versions.extend([12,3])

In [22]:
model_version_policy.WhichOneof('policy_choice')

'specific'

In [27]:
model_version_policy.all.SetInParent()

In [28]:
model_version_policy.WhichOneof('policy_choice')

'all'

In [17]:
policy_choice = model_version_policy.WhichOneof('policy_choice')

In [ ]:
def model_version_policy(self, _policy):
    policy = file_system_storage_path_source_pb2.FileSystemStoragePathSourceConfig.ServableVersionPolicy() 
    if isinstance(_policy, dict):
        for k, v in _policy.items():
            if k == 'latest':
                policy.latest.num_versions = v
            elif k == 'specific':
                if not isinstance(v, (list, tuple)):
                    v = [v]
                policy.specific.extend(v)
    elif isinstance(_policy, str):
        if _policy == 'all':
            policy.all.SetInParent()
    else:
        raise AttributeError('Unknown value for attribute `model_version_policy`.')
    self._protobuf.model_version_policy.CopyFrom(policy)

In [19]:
policy = {'all':None}

In [20]:
policy

{'all': None}

In [ ]:
if policy_choice == 'latest':
    model_version_policy.latest.num_versions = 1
if policy_choice == 'specific':
    # provide a list of specific version to host
    pass
if policy_choice == 'all':
    # just set in parent
    passa
    

In [1]:
from config import ModelConfig

In [6]:
model_config = ModelConfig(name='mnist', base_path='./save', model_version_policy='all', version_labels={'s':1, 't':2})

In [7]:
model_config.logging_config

In [4]:
model_config

name: "mnist"
base_path: "./save"
model_version_policy {
  all {
  }
}
version_labels {
  key: "s"
  value: 1
}
version_labels {
  key: "t"
  value: 2
}